### Imports

In [1]:
!pip install js2py
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from datetime import datetime
from bokeh.palettes import Spectral3
from tabulate import tabulate
from bokeh.transform import factor_cmap
import js2py
import math
from concurrent.futures import ProcessPoolExecutor as PoolExecutor
import requests

# convenience
output_notebook()
pd.set_option('display.max_colwidth', -1)

Loading BokehJS ...

### Globals

In [2]:
API = 'http://52.53.163.242/api/'
NUM_CHALLENGES = 100
NUM_BLOCKS = 100
TOP = 10
animalhash = js2py.require('angry-purple-tiger')
MAX_WORKERS = 8
REWARD_LIMIT = 500

### Functions

In [3]:
def latest_height(api):
    '''
    Get latest block height
    '''
    r = requests.get(f'{api}/blocks?limit=1')
    if r.status_code == 200:
        data = r.json().get('data')
        return data[0]['height']
    raise Exception('Unable to get latest height')

def get_blocks(api, num_blocks=100):
    '''
    Get past `NUM_BLOCKS` number of num_blocks
    '''
    r = requests.get(f'{api}/blocks?limit={num_blocks}')
    if r.status_code == 200:
        data = r.json().get('data')
        assert len(data) == num_blocks
        return data
    raise Exception('Unable to get blocks')

def get_challenges(api, num_challenges=100):
    '''
    Get past `LIMIT` number of challenges
    '''
    r = requests.get(f'{api}/challenges?limit={num_challenges}')
    if r.status_code == 200:
        data = r.json().get('data')
        assert len(data) == num_challenges
        return accumulate_results(data)
    raise Exception('Unable to get challenges')

def get_hotspots(api):
    '''
    Get hotspots from the api
    '''
    r = requests.get(f'{api}/hotspots')
    if r.status_code == 200:
        data = r.json().get('data')
        return data
    raise Exception('Unable to get hotspots')

def get_accounts(api):
    '''
    Get accounts from the api
    '''
    r = requests.get(f'{api}/accounts')
    if r.status_code == 200:
        data = r.json().get('data')
        return data
    raise Exception('Unable to get accounts')

def accumulate_results(challenges):
    '''
    Return a dict list of challenge_id, num_successes, num_failures and num_untested
    '''
    results = []
    for challenge in challenges:
        path = challenge['pathElements']

        result = [element['result'] for element in path]
        length = len(result)
        num_untested = result.count('untested')

        gray = True if num_untested == length else False

        if gray:
            num_successes = 0
            num_failures = 0
            num_untested = 0
            dud = 1
            success = 0
        else:
            num_successes = result.count('success')
            num_failures = result.count('failure')
            num_untested = num_untested
            dud = 0
            success = 1 if num_successes == length else 0
        results.append({'id': challenge['id'],
                        'num_successes': num_successes,
                        'num_failures': num_failures,
                        'num_untested': num_untested,
                        'dud': dud,
                        'success': success})
    return results


def get_reward(reward_url):
    '''
    Get hotspot reward
    '''
    r = requests.get(reward_url)
    if r.status_code == 200:
        data = r.json().get('data')
        return data
    raise Exception(f'Unable to get rewards for {hotspot}')
    
def get_rewards(api, hotspots, limit=REWARD_LIMIT):
    '''
    Get rewards for all hotspots
    '''
    reward_urls = [f'{api}/hotspots/{hotspot}/rewards?limit={limit}' for hotspot in hotspots]
    with PoolExecutor(max_workers=MAX_WORKERS) as executor:
        rewards = []
        for hotspot, i in zip(hotspots, executor.map(get_reward, reward_urls)):
            rewards.extend(i)
    return rewards

### Block Stats

In [4]:
blocks = get_blocks(API, NUM_BLOCKS)

In [5]:
df = pd.DataFrame(blocks)

In [6]:
print(f'Average txns past {NUM_BLOCKS} blocks: {df["txns"].mean()}')

Average txns past 100 blocks: 4.66


In [7]:
print(f'Average block time past {NUM_BLOCKS} blocks: {df["time"].mean()}')

Average block time past 100 blocks: 1565643385.16


In [8]:
max_txns = tabulate(df.max().reset_index(),tablefmt='psql', showindex='never')
print(f"Block with max txns for past {NUM_BLOCKS} blocks: \n{max_txns}")

Block with max txns for past 100 blocks: 
+--------+----------------------------------------------------+
| hash   | 1z96KEE8y9srrcfVbVs3gU8MLWKosuPu31EqKztswmKVrhadcv |
| height | 14444                                              |
| round  | 14444                                              |
| time   | 1565649527                                         |
| txns   | 13                                                 |
+--------+----------------------------------------------------+


In [9]:
source = ColumnDataSource(df)
p = figure()

p.vbar(x='height', top='txns', width=0.8, source=source)
hover = HoverTool(tooltips=[('height', '@height'), ('txns', '@txns')])
p.add_tools(hover)
p.xaxis.axis_label = "Txns/Block"
p.yaxis.axis_label = "NumTxns"

show(p)

### Hotspot Stats

In [10]:
hotspots = get_hotspots(API)

In [11]:
hdf = pd.DataFrame(hotspots)
hdf['name'] = hdf.apply(lambda row: '-'.join(animalhash(row.address).lower().split()), axis=1)

In [12]:
print(f'Top {TOP} scoring hotspots:')
hdf.sort_values('score', ascending=False)[['name', 'score']].head(TOP)

Top 10 scoring hotspots:


,name,score
112,magic-carob-quail,0.9901
9,beautiful-onyx-platypus,0.9404
129,mean-alabaster-piranha,0.9389
99,delightful-stone-beetle,0.9342
126,mini-currant-lizard,0.9126
128,mythical-tin-hawk,0.9064
56,shambolic-rusty-cyborg,0.9010
122,joyous-burgundy-chicken,0.8942
40,bubbly-mandarin-panther,0.8779
68,teeny-cider-troll,0.8307


In [13]:
source = ColumnDataSource(hdf)
# p = figure()
p = figure(x_range=hdf['name'], y_range=[0,1], title="Hotspot Scores")

p.vbar(x='name', top='score', width=0.8, source=source)
hover = HoverTool(tooltips=[('name', '@name'), ('score', '@score')])
p.width = 900
p.add_tools(hover)
p.xaxis.axis_label = "Name"
p.yaxis.axis_label = "Score"
p.xaxis.major_label_text_font_size = "4pt"
p.xaxis.major_label_orientation = math.pi/4

show(p)

### Account stats

In [14]:
accounts = get_accounts(API)

In [15]:
df = pd.DataFrame(accounts)

In [16]:
print(f'Top {TOP} account balances:')
df.sort_values('balance', ascending=False)[['address', 'balance']].head(TOP)

Top 10 account balances:


,address,balance
39,13jL4pTLq4JQBwDoH1ZBX3VPchmeTZxQSGi4cnyXXuzPPoKxcd5,4528288338848
66,1398hLeHESZHE5jVtaLAV5fdg2vrUeZEs2B92t7TzeQTtugr8dL,3148556649711
94,14a5sytsEVKBVhGsVz2rJDkxf1cs1L89zvtFjmPtmPzrQHXPVjF,2589935030203
79,14GWyFj9FjLHzoN3aX7Tq7PL6fEg4dfWPY8CrK8b9S5ZrcKDz6S,1490327444446
82,12yyntxFfPhqXDKuEvmAWABNTRnXtYmcgS33WN4tk5YtyaBLDXo,1488478189484
46,13o9KUMcmEdhtjXRaJ5AANHLZWBoGcTmF4MFw3MjQbubjR57jXW,1346536530761
78,12z4nUiayZnbW46azcegBBB9yGkchFb1Zm7EPuh6eV1r2HT1NgC,906696666770
72,14i6nM1RbWbjQFg5NFbZS3aZJgpUGZUx5toarz6jZLPpNAb95Tq,854254505662
86,14UgUhAPq53iR8vfde6sU4vLPWRe6kZ3ioV7xaVb8hr3oafaok6,851615595904
75,135bB6B7gGrsdPNcFwvcn12KPjTktSNZvmVbvSBKubCtEgxkpPJ,827031020581


### Challenge Stats

In [17]:
challenges = get_challenges(API, NUM_CHALLENGES)

In [18]:
df = pd.DataFrame(challenges)

In [19]:
print(f"Total Successful challenges past {NUM_CHALLENGES} challenges: {sum(df['success'] == 1)}")

Total Successful challenges past 100 challenges: 23


In [20]:
print(f"Entirely dud challenges since past {NUM_CHALLENGES} challenges: {sum(df['dud'] == 1)}")

Entirely dud challenges since past 100 challenges: 14


### Hotspot Reward Stats

In [21]:
rewards = get_rewards(API, hdf['address'].values)

In [22]:
rdf = pd.DataFrame(rewards)[['gateway', 'amount']]

In [23]:
g1 = rdf.groupby('gateway').sum().reset_index()
g1['name'] = g1.apply(lambda row: '-'.join(animalhash(row.gateway).lower().split()), axis=1)

In [24]:
print(f'Top {TOP} hotspot rewards:')
g1.sort_values('amount', ascending=False)[['name', 'amount']].head(TOP)

Top 10 hotspot rewards:


,name,amount
104,shambolic-rusty-cyborg,1289992390388
53,mean-alabaster-piranha,835212477515
8,mythical-tin-hawk,762352554214
89,clean-nylon-okapi,746384961112
25,fit-iron-monkey,578934343103
45,cuddly-sapphire-narwhal,527163119031
116,bubbly-mandarin-panther,524531786496
95,delightful-stone-beetle,480325866710
73,magic-carob-quail,477675743467
109,teeny-cider-troll,472687560339


In [25]:
source = ColumnDataSource(g1)
p = figure(x_range=g1['name'], title="Hotspot Rewards")

p.vbar(x='name', top='amount', width=0.8, source=source)
hover = HoverTool(tooltips=[('name', '@name'), ('amount', '@amount')])
p.width = 900
p.add_tools(hover)
p.xaxis.axis_label = "Name"
p.yaxis.axis_label = "Reward"
p.xaxis.major_label_text_font_size = "4pt"
p.xaxis.major_label_orientation = math.pi/4

show(p)